<a href="https://colab.research.google.com/github/DVerma11/Reddit_Anxiety_Symptoms_Narratives_NLP_Exploration/blob/main/Part_2_Data_Preprocessing_on_Raw_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part2: Data Preprocessing on raw file


## 2.1 Import necessary libraries and models











In [ ]:
%%capture
pip install praw

In [ ]:
import praw
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import classification_report
from spacy.matcher import Matcher

In [ ]:
#install libraries
%%capture
!pip install pandas

In [ ]:
# Import pandas for data manipulation
import pandas as pd

In [ ]:
# Core NLP + utilities
!pip install pandas numpy spacy langdetect beautifulsoup4 emoji

# Scientific spaCy models
!pip install scispacy

# Download spaCy model (general-purpose)
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 17.9 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=e93a78a97a2b594a688f8ce9cfc168ee6b323af8b3cc25d35dd14582d9e9ab04
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 25.4 MB/s eta 0:00:00
  Attempting uninstall: nu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 68.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Restart session


In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import emoji
from langdetect import detect
import spacy
from datetime import datetime
from IPython.display import FileLink, display
import numpy as np


## 2.2 Load Data downloaded from Reddit






In [ ]:
import pandas as pd

file_path = "reddit_anxietysymptoms_comments_raw.csv"

anxiety_symptoms_raw_df= pd.read_csv(
    file_path,
    encoding="utf-8",
    low_memory=False
)


## 2.3 Explore Data

In [ ]:
# --- Function to explore a dataset ---
def explore_text_dataset(df, name, text_col='full_text'):
    print(f"\n=== {name} Dataset ===")
    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist())

    # Unique IDs (if exist)
    id_cols = [col for col in ['post_id', 'comment_id'] if col in df.columns]
    if id_cols:
        print("Unique IDs:")
        for col in id_cols:
            print(f"  {col}: {df[col].nunique()} unique")

    # Author hash completeness
    if "author_hash" in df.columns:
        total = len(df)
        missing = df["author_hash"].isna().sum() + (df["author_hash"].astype(str).str.strip() == "").sum()
        present = total - missing

        print("\nAuthor hash completeness:")
        print(f"  Present: {present:,} ({present/total:.2%})")
        print(f"  Missing: {missing:,} ({missing/total:.2%})")

    # Text length stats
    if text_col in df.columns:
        print("\nText column stats:", text_col)
        print("  Empty/missing:", df[text_col].isna().sum())
        print("  Blank strings:", (df[text_col].str.strip() == '').sum())
        print("  Character length stats:")
        print(df[text_col].astype(str).str.len().describe())
        print("  Newline counts per record:")
        print(df[text_col].str.count('\n').describe())
        # Sample text
        print("\nSample text:")
        print(df[[text_col]].sample(3, random_state=42))

        # Optional: normalize text for NLP
        df[f'{text_col}_clean'] = df[text_col].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()

# --- Run exploration ---
explore_text_dataset(anxiety_symptoms_raw_df, "Symptoms", text_col='full_text')




=== Symptoms Dataset ===
Shape: (3822, 10)
Columns: ['post_id', 'comment_id', 'parent_id', 'comment_body', 'author_hash', 'score', 'created_utc', 'title', 'post_body', 'full_text']
Unique IDs:
  post_id: 14 unique
  comment_id: 3822 unique

Author hash completeness:
  Present: 3,418 (89.43%)
  Missing: 404 (10.57%)

Text column stats: full_text
  Empty/missing: 0
  Blank strings: 0
  Character length stats:
count     3822.000000
mean      2777.149137
std       3707.687061
min         49.000000
25%        324.000000
50%        719.500000
75%       4664.750000
max      19016.000000
Name: full_text, dtype: float64
  Newline counts per record:
count    3822.000000
mean       27.991104
std        36.018423
min         0.000000
25%         2.000000
50%         4.000000
75%        62.000000
max       184.000000
Name: full_text, dtype: float64

Sample text:
                                              full_text
3194  What are your physical symptoms of chronic anx...
3708  What does Generalis

## 2.4 Remove blank author hash

In [ ]:
before = len(anxiety_symptoms_raw_df)

anxiety_symptoms_raw_df = anxiety_symptoms_raw_df[
    anxiety_symptoms_raw_df["author_hash"].notna() &
    (anxiety_symptoms_raw_df["author_hash"].str.strip() != "")
]

after = len(anxiety_symptoms_raw_df)

print(f"Removed {before - after} rows with missing author_hash")
print(f"Symptoms: after removing rows with missing author → {len(anxiety_symptoms_raw_df)} rows")


Removed 404 rows with missing author_hash
Symptoms: after removing rows with missing author → 3418 rows


## 2.5 Language Filter

 Keep "english" language; Remove deleted / removed posts; Remove promotional / survey posts; remove exact duplicates; Keep certain comments only >25 percentile of upvotes


In [ ]:
def language_filter(text):
    try:
        return detect(text) == 'en'
    except Exception:
        return False
anxiety_symptoms_raw_df = anxiety_symptoms_raw_df[anxiety_symptoms_raw_df["comment_body"].apply(language_filter)]
print(f"Symptoms: after language filtering → {len(anxiety_symptoms_raw_df)} rows")



print(anxiety_symptoms_raw_df.shape)



Symptoms: after language filtering → 3248 rows
(3248, 11)


## 2.6 Remove deleted / removed comments

In [ ]:
def is_valid_text(text):
    if not isinstance(text, str):
        return False
    text_lower = text.lower().strip()
    return text_lower not in ["[deleted]", "[removed]"]

anxiety_symptoms_raw_df = anxiety_symptoms_raw_df[anxiety_symptoms_raw_df["comment_body"].apply(is_valid_text)].copy()


print(f"Symptoms dataset after dropping deleted/removed comments: {len(anxiety_symptoms_raw_df)} rows")



Symptoms dataset after dropping deleted/removed comments: 3248 rows


## 2.7 Remove promotional / survey comments

In [ ]:
promo_keywords = [
    "survey",
    "questionnaire",
    "study link",
    "research study",
    "click here",
    "sign up"
]
promo_pattern = "|".join(promo_keywords)

anxiety_symptoms_raw_df = anxiety_symptoms_raw_df[~anxiety_symptoms_raw_df["comment_body"].str.lower().str.contains(promo_pattern, na=False)]


print(f"Symptoms dataset after dropping promotional/survey posts: {len(anxiety_symptoms_raw_df)} rows")



Symptoms dataset after dropping promotional/survey posts: 3248 rows


## 2.8 Remove exact duplicates

In [ ]:
anxiety_symptoms_raw_df = anxiety_symptoms_raw_df.drop_duplicates(subset=["comment_body"]).reset_index(drop=True)

print(f"Symptoms dataset after dropping duplicates: {len(anxiety_symptoms_raw_df)} rows")

anxiety_symptoms_raw_df = anxiety_symptoms_raw_df.drop_duplicates(subset=["comment_body"]).reset_index(drop=True)



Symptoms dataset after dropping duplicates: 3210 rows


## 2.9 Keep >25th percentile of upvotes

In [ ]:
#Upvotes reflect community validation of meaning, not surface form
#This mirrors best practice in social media NLP papers
def filter_top_25(df, score_col='score'):
    threshold = df[score_col].quantile(0.25)
    filtered_df = df[df[score_col] >= threshold].copy()
    filtered_df.reset_index(drop=True, inplace=True)
    return filtered_df

anxiety_symptoms_raw_df = filter_top_25(anxiety_symptoms_raw_df, score_col='score')

# ------------------------------
#  Print final counts
# ------------------------------
print(f"Symptoms dataset after keeping >25th percentile of upvotes: {len(anxiety_symptoms_raw_df)} rows")


Symptoms dataset after keeping >25th percentile of upvotes: 3196 rows


## 2.10 Light cleaning with emoji mapping

In [ ]:
!pip install unidecode


In [ ]:
from bs4 import BeautifulSoup
import re
import emoji
from unidecode import unidecode

# Emoji mapping
emoji_to_text = {
    "😊": "happy",
    "😃": "happy",
    "😂": "laugh",
    "😢": "sad",
    "😡": "angry",
    "🤯": "shocked",
    "🤩": "excited",
    "😞": "sad",
    "😔": "sad",
    "😱": "fear",
    "❤️": "love",
}

def emoji_to_word(text):
    for e, word in emoji_to_text.items():
        text = text.replace(e, f" {word} ")
    return text

def light_clean(text):
    if not isinstance(text, str):
        return ""
    text = BeautifulSoup(text, "html.parser").get_text() # Remove HTML
    text = unidecode(text) # Convert weird unicode like â€™ or Â to plain ASCII
    text = text.replace("\n", " ") # Replace newlines with spaces
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
    text = emoji_to_word(text)# Replace emojis with words
    text = re.sub(r"[^a-zA-Z\s\.\!\?\,\;\:\(\)\-]", " ", text) # Keep letters, spaces, and useful punctuation for NLP
    text = re.sub(r"\s+", " ", text).strip()   # Normalize whitespace
    text = text.lower() # Lowercase
    text = re.sub(r"\s*([.!?])\s*", r"\1 ", text) # Clean spacing around sentence-ending punctuation
    return text


In [ ]:
# Clean comment_body
anxiety_symptoms_raw_df["comment_body_clean"] = anxiety_symptoms_raw_df["comment_body"].apply(light_clean)

# Clean post_body
anxiety_symptoms_raw_df["post_body_clean"] = anxiety_symptoms_raw_df["post_body"].apply(light_clean)



In [ ]:
# Preview
anxiety_symptoms_raw_df[["author_hash", "comment_body", "comment_body_clean", "post_body", "post_body_clean"]].head(5)


,author_hash,comment_body,comment_body_clean,post_body,post_body_clean
0,bfc763f738dd81303e35d089fde639e68495eab77cc322...,omg you have no idea how much better i feel. i...,omg you have no idea how much better i feel. i...,Anxiety easily can cause a million different s...,anxiety easily can cause a million different s...
1,9bba55d20948ae8babbea1c68977c6d0c65cfc5a6d7412...,Thanks for this. I experience a ton of similar...,thanks for this. i experience a ton of similar...,Anxiety easily can cause a million different s...,anxiety easily can cause a million different s...
2,4c07dfd4a1c67f96a0e9814edb0c983b564ff788ed7e64...,Though feeling all of these symptoms is incred...,though feeling all of these symptoms is incred...,Anxiety easily can cause a million different s...,anxiety easily can cause a million different s...
3,8d501e41f5ffc02c9d373eaf3705a596714dbafb7531de...,Wow this was so reassuring to me. I was just t...,wow this was so reassuring to me. i was just t...,Anxiety easily can cause a million different s...,anxiety easily can cause a million different s...
4,0814aba0673db4ca9b746ba9f82b8bf3bfb671f48d7d9b...,Hello everyone. I’m currently recovering from ...,hello everyone. i m currently recovering from ...,Anxiety easily can cause a million different s...,anxiety easily can cause a million different s...


In [ ]:
# Preview first 10 rows of all columns
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", 200)         # Make wide display easier
anxiety_symptoms_raw_df.head(10)


,post_id,comment_id,parent_id,comment_body,author_hash,score,created_utc,title,post_body,full_text,full_text_clean,comment_body_clean,post_body_clean
0,1czzuoo,l5k13qf,t3_1czzuoo,omg you have no idea how much better i feel. i...,bfc763f738dd81303e35d089fde639e68495eab77cc322...,110,1.716601e+09,Here is a full list of anxiety symptoms I deal...,Anxiety easily can cause a million different s...,Here is a full list of anxiety symptoms I deal...,Here is a full list of anxiety symptoms I deal...,omg you have no idea how much better i feel. i...,anxiety easily can cause a million different s...
1,1czzuoo,l5k4qae,t3_1czzuoo,Thanks for this. I experience a ton of similar...,9bba55d20948ae8babbea1c68977c6d0c65cfc5a6d7412...,43,1.716603e+09,Here is a full list of anxiety symptoms I deal...,Anxiety easily can cause a million different s...,Here is a full list of anxiety symptoms I deal...,Here is a full list of anxiety symptoms I deal...,thanks for this. i experience a ton of similar...,anxiety easily can cause a million different s...
2,1czzuoo,l5l1d2d,t3_1czzuoo,Though feeling all of these symptoms is incred...,4c07dfd4a1c67f96a0e9814edb0c983b564ff788ed7e64...,11,1.716622e+09,Here is a full list of anxiety symptoms I deal...,Anxiety easily can cause a million different s...,Here is a full list of anxiety symptoms I deal...,Here is a full list of anxiety symptoms I deal...,though feeling all of these symptoms is incred...,anxiety easily can cause a million different s...
3,1czzuoo,l5kjm8b,t3_1czzuoo,Wow this was so reassuring to me. I was just t...,8d501e41f5ffc02c9d373eaf3705a596714dbafb7531de...,7,1.716610e+09,Here is a full list of anxiety symptoms I deal...,Anxiety easily can cause a million different s...,Here is a full list of anxiety symptoms I deal...,Here is a full list of anxiety symptoms I deal...,wow this was so reassuring to me. i was just t...,anxiety easily can cause a million different s...
4,1czzuoo,lwwknq7,t3_1czzuoo,Hello everyone. I’m currently recovering from ...,0814aba0673db4ca9b746ba9f82b8bf3bfb671f48d7d9b...,8,1.731499e+09,Here is a full list of anxiety symptoms I deal...,Anxiety easily can cause a million different s...,Here is a full list of anxiety symptoms I deal...,Here is a full list of anxiety symptoms I deal...,hello everyone. i m currently recovering from ...,anxiety easily can cause a million different s...
5,1czzuoo,l5kq0sj,t3_1czzuoo,So many of these symptoms I didn't realize had...,50657c1edc09332f8fa1b89daad4842290e95a24665696...,7,1.716614e+09,Here is a full list of anxiety symptoms I deal...,Anxiety easily can cause a million different s...,Here is a full list of anxiety symptoms I deal...,Here is a full list of anxiety symptoms I deal...,so many of these symptoms i didn t realize had...,anxiety easily can cause a million different s...
6,1czzuoo,l5lcw8u,t3_1czzuoo,Thanks for this I have severe health anxiety c...,33441958717941b8595811e0e5f8300033bd95178be9dd...,7,1.716631e+09,Here is a full list of anxiety symptoms I deal...,Anxiety easily can cause a million different s...,Here is a full list of anxiety symptoms I deal...,Here is a full list of anxiety symptoms I deal...,thanks for this i have severe health anxiety c...,anxiety easily can cause a million different s...
7,1czzuoo,l5leb2f,t3_1czzuoo,Thanks very much for this. I always suffered a...,d9f385c0de02e5eed19225014b39ac1cd2097eb837b9f9...,6,1.716632e+09,Here is a full list of anxiety symptoms I deal...,Anxiety easily can cause a million different s...,Here is a full list of anxiety symptoms I deal...,Here is a full list of anxiety symptoms I deal...,thanks very much for this. i always suffered a...,anxiety easily can cause a million different s...
8,1czzuoo,l5lmehb,t3_1czzuoo,Thank you for this comprehensive list of your ...,acfe66793666bb86852f22fc001ef42a6bc988cf4f91e4...,7,1.716637e+09,Here is a full list of anxiety symptoms I deal...,Anxiety easily can cause a million different s...,Here is a full list of anxiety symptoms I deal...,

## 2.11 Save preprocessed file: symptoms_cleaned (with clean_text column)

In [ ]:
# Symptoms
anxiety_symptoms_raw_df.to_csv("Step1_reddit_anxiety_preprocessed.csv", index=False, encoding="utf-8")


End of Part 2